#### Define Imports

In [1]:
#%pip install openai

In [1]:
import pandas as pd 
from sklearn.model_selection import train_test_split
import json
import os
import openai
from openai import OpenAI
import time
import sys

sys.path.insert(0,'../')
from environment import env
from nps_model_functions import helper_functions

#### Define Variables

In [2]:
config = env.env()

# Inputs
client = OpenAI(api_key  = config['gpt_api_key'])
root = config['root']
syn_queries = root+'02_nps_api_data\\synthetic_queries.csv'

# Output
model_name = 'nps_model_parkcode'
train_data = 'parkcode_train_data.jsonl'
val_data = 'parkcode_val_data.jsonl'
target = 'parkcode'

#### Read Inputs

In [3]:
synthetic_queries_df = pd.read_csv(syn_queries)

#### Prepare for GPT

In [5]:
train_df, val_df = train_test_split(synthetic_queries_df, test_size=0.2, random_state=42)
print(len(train_df),len(val_df))

helper_functions.save_to_jsonl(train_df, train_data, target)
helper_functions.save_to_jsonl(val_df, val_data, target)

#### Fine Tune

In [6]:
train_file =  client.files.create(
  file=open("parkcode_train_data.jsonl", "rb"),
  purpose="fine-tune"
)

val_file = client.files.create(
  file=open("parkcode_val_data.jsonl", "rb"),
  purpose="fine-tune"
)

train_file_id = train_file.id
val_file_id = val_file.id

In [7]:
fine_tune = client.fine_tuning.jobs.create(
    model="davinci-002",
    training_file=train_file_id,
    validation_file=val_file_id,
    seed = 42,
    suffix = model_name
)
fine_tune_id = fine_tune.id